# 112 feature

In [1]:
import numpy as np
import scanpy as sc
import anndata as ad
import multimil as mtm
import numpy as np
import scanpy as sc
import scvi
import warnings

warnings.filterwarnings("ignore")

[rank: 0] Global seed set to 0


In [2]:
# !pip list

In [3]:
import datasets
import pandas as pd

meta_df = pd.read_csv("/home/tl688/pitl688/hest_data/HEST_v1_1_0.csv", index_col=0)

In [4]:

# meta_df

In [5]:
meta_df_t = meta_df[meta_df['st_technology'] == 'Visium']
meta_df_t = meta_df_t[meta_df_t['species'] == 'Homo sapiens']

In [6]:
meta_df_t['disease_state'].value_counts()

disease_state
Cancer      172
Diseased    114
Healthy     102
Treated      26
Tumor         7
Name: count, dtype: int64

In [7]:
meta_df_t['disease_state'] = [str(i) for i in meta_df_t['disease_state'].values]
meta_df_t['oncotree_code'] = [str(i) for i in meta_df_t['oncotree_code'].values]

In [8]:
disease_update = []
for i in meta_df_t.index:
    if meta_df_t.loc[i]['oncotree_code'] != 'nan':
        disease_update.append(meta_df_t.loc[i]['oncotree_code'])
    else:
        disease_update.append(meta_df_t.loc[i]['disease_state'])

In [9]:
meta_df_t['disease_update'] = disease_update 

In [10]:
set(meta_df_t['disease_update'] )

{'ACYC',
 'BLCA',
 'CESC',
 'COAD',
 'COADREAD',
 'Cancer',
 'Diseased',
 'EPM',
 'GBM',
 'HCC',
 'HGSOC',
 'Healthy',
 'IDC',
 'ILC',
 'LNET',
 'LUSC',
 'PAAD',
 'PRAD',
 'READ',
 'SCCRCC',
 'SOC',
 'Treated',
 'Tumor',
 'UNKNOWN'}

In [11]:
meta_df_t.index = [i for i in range(len(meta_df_t))]

In [12]:
meta_df_t_update = meta_df_t[meta_df_t['disease_update'].isin(['ACYC',
 'BLCA',
 'CESC',
 'COAD',
 'COADREAD',
 'EPM',
 'GBM',
 'HCC',
 'HGSOC',
 'Healthy',
 'IDC',
 'ILC',
 'LNET',
 'LUSC',
 'PAAD',
 'PRAD',
 'READ',
 'SCCRCC',
 'SOC',])]

In [13]:
meta_df_t_update

,dataset_title,id,image_filename,organ,disease_state,oncotree_code,species,patient,st_technology,data_publication_date,...,pixel_size_um_embedded,pixel_size_um_estimated,magnification,fullres_px_width,fullres_px_height,tissue,disease_comment,subseries,hest_version_added,disease_update
0,Characterization of immune cell populations in...,TENX152,TENX152.tif,Bowel,Cancer,COAD,Homo sapiens,Patient 1,Visium,7/11/24,...,0.264583,0.273724,40x,76190,49099,Colon,NaN,"Visium CytAssist v2, Sample P2 CRC",v1_1_0,COAD
1,Spatially resolved multiomics of human cardiac...,MISC142,MISC142.tif,Heart,Healthy,nan,Homo sapiens,,Visium,NaN,...,31.750063,0.454959,40x,18063,18100,right cardiac atrium,NaN,HCAHeartST10238251,v1_1_0,Healthy
2,Spatially resolved multiomics of human cardiac...,MISC141,MISC141.tif,Heart,Healthy,nan,Homo sapiens,,Visium,NaN,...,31.750063,0.455062,40x,17890,18098,left cardiac atrium,NaN,HCAHeartST10238252,v1_1_0,Healthy
3,Spatially resolved multiomics of human cardiac...,MISC140,MISC140.tif,Heart,Healthy,nan,Homo sapiens,,Visium,NaN,...,31.750063,0.455235,40x,17802,18044,apical region of left ventricle,NaN,HCAHeartST10238253,v1_1_0,Healthy
4,Spatially resolved multiomics of human cardiac...,MISC139,MISC139.tif,Heart,Healthy,nan,Homo sapiens,,Visium,NaN,...,31.750063,0.455581,40x,17634,17674,heart right ventricle,NaN,HCAHeartST10550730,v1_1_0,Healthy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,spatialLIBD,MISC5,MISC5.tif,Brain,Healthy,nan,Homo sapiens,,Visium,NaN,...,NaN,0.727113,20x,13332,13332,dorsolateral prefrontal cortex,NaN,151672,v1_0_0,Healthy
417,spatialLIBD,MISC4,MISC4.tif,Brain,Healthy,nan,Homo sapiens,,Visium,NaN,...,NaN,0.726109,20x,13332,13332,dorsolateral prefrontal cortex,NaN,151673,v1_0_0,Healthy
418,spatialLIBD,MISC3,MISC3.tif,Brain,Healthy,nan,Homo sapiens,,Visium,NaN,...,NaN,0.725124,20x,13332,13332,dorsolateral prefrontal cortex,NaN,151674,v1_0_0,Healthy
419,spatialLIBD,MISC2,MISC2.tif,Brain,Healthy,nan,Homo sapiens,,Visium,NaN,...,NaN,0.726109,20x,13332,13332,dorsolateral prefrontal cortex,NaN,151675,v1_0_0,Healthy


In [14]:
meta_df[meta_df['id'].isin(["TENX111", "TENX149", "TENX148","TENX147"])]

,dataset_title,id,image_filename,organ,disease_state,oncotree_code,species,patient,st_technology,data_publication_date,...,treatment_comment,pixel_size_um_embedded,pixel_size_um_estimated,magnification,fullres_px_width,fullres_px_height,tissue,disease_comment,subseries,hest_version_added
8,Characterization of immune cell populations in...,TENX149,TENX149.tif,Bowel,Cancer,COAD,Homo sapiens,Patient 1,Xenium,7/11/24,...,NaN,0.273777,0.274031,40x,70534,60222,Colon,Stage II-A,"Xenium In Situ, Sample P1 CRC",v1_1_0
9,Characterization of immune cell populations in...,TENX148,TENX148.tif,Bowel,Cancer,COAD,Homo sapiens,Patient 1,Xenium,7/11/24,...,NaN,0.273776,0.273760,40x,75184,48582,Colon,NaN,"Xenium In Situ, Sample P2 CRC",v1_1_0
10,Characterization of immune cell populations in...,TENX147,TENX147.tif,Bowel,Cancer,COAD,Homo sapiens,Patient 1,Xenium,7/11/24,...,NaN,0.273775,0.249702,40x,71082,66226,Colon,Stage IV-A,"Xenium In Situ, Sample P5 CRC",v1_1_0
149,Human Colon Preview Data (Xenium Human Colon G...,TENX111,TENX111.tif,Bowel,Cancer,COAD,Homo sapiens,NaN,Xenium,8/29/23,...,NaN,0.212500,0.273805,40x,26909,43090,Colon,cancer; stage 2A adenocarcinoma,"Cancer, pre-designed + add-on panel",v1_0_0


In [15]:
meta_df[meta_df['id'] == "TENX111"]

,dataset_title,id,image_filename,organ,disease_state,oncotree_code,species,patient,st_technology,data_publication_date,...,treatment_comment,pixel_size_um_embedded,pixel_size_um_estimated,magnification,fullres_px_width,fullres_px_height,tissue,disease_comment,subseries,hest_version_added
149,Human Colon Preview Data (Xenium Human Colon G...,TENX111,TENX111.tif,Bowel,Cancer,COAD,Homo sapiens,NaN,Xenium,8/29/23,...,NaN,0.2125,0.273805,40x,26909,43090,Colon,cancer; stage 2A adenocarcinoma,"Cancer, pre-designed + add-on panel",v1_0_0


In [17]:
adata = sc.read("/home/tl688/scratch/adata_imp_disease_update.h5ad")

In [29]:
result_list_new = []
kf = KFold(n_splits=3, shuffle=True, random_state=2024)
for train, test in kf.split(X):
    train_batch = X[train]
    train_label = meta_df_t_update.loc[train_batch]['disease_update'].values
#     model = xgboost.XGBClassifier(n_jobs=-1)
    
#     enc = sklearn.preprocessing.LabelEncoder()
#     train_label = enc.fit_transform(train_label)

    model = sklearn.linear_model.LogisticRegression(n_jobs=-1, max_iter=1000)
    
    model.fit(train_data[train_batch], train_label)
    
    test_batch = X[test]
    test_label = meta_df_t_update.loc[test_batch]['disease_update'].values
    
    pred = model.predict(train_data[test_batch])
#     pred = enc.inverse_transform(pred)
    
    dict_out = sklearn.metrics.classification_report(test_label,pred,digits=4, output_dict=True)
    single_out = [dict_out['accuracy'], dict_out['weighted avg']['precision'] , dict_out['weighted avg']['recall'] , dict_out['weighted avg']['f1-score']]
    result_list_new.append(single_out)

In [30]:
for i in result_list_new:
    print(i[0], ' ', i[1], ' ', i[2], ' ', i[3])

0.43037974683544306   0.3630379746835443   0.43037974683544306   0.37762740866048333
0.2692307692307692   0.35413051860420286   0.2692307692307692   0.3023855083556576
0.34615384615384615   0.30912780912780913   0.34615384615384615   0.32413882413882417
